In [143]:
import re
import json

In [144]:
class ParsingError(Exception):
    """Custom exception for parsing errors."""
    def __init__(self, message, position=None):
        self.message = message
        self.position = position
        super().__init__(message)

In [145]:
class MTddVParser:
    def __init__(self, code):
        self.code = code
        self.tokens = self.tokenize(code)
        self.current_token_index = 0
        self.errors = []
        self.boucle_stack = []  # Stack to track nested 'boucle' blocks

    def tokenize(self, code):
        """Tokenize the input code by splitting on whitespace and special characters."""
        tokens = re.findall(r'\w+|\(|\)|\}|fin|#|%', code)
        return tokens

    def current_token(self):
        if self.current_token_index < len(self.tokens):
            return self.tokens[self.current_token_index]
        return None

    def advance(self):
        """Move to the next token."""
        self.current_token_index += 1

    def parse(self):
        structure = []
        while self.current_token():
            try:
                instruction = self.parse_instruction()
                if instruction:
                    structure.append(instruction)
            except ParsingError as e:
                self.errors.append({"error": e.message, "position": self.current_token_index})
                # Skip to the next token to attempt recovery
                self.advance()

        # Check for unclosed 'boucle' blocks
        if self.boucle_stack:
            self.errors.append({"error": "Unclosed 'boucle' block(s) detected.", "position": self.current_token_index})

        # Check if the last token is '#'
        if self.tokens and self.tokens[-1] != "#":
            self.errors.append({"error": "Missing '#' at the end of the program.", "position": len(self.tokens) - 1})

        return structure

    def parse_instruction(self):
        token = self.current_token()

        # Basic Instructions
        if token in ["I", "P", "G", "D", "0", "1"]:
            self.advance()
            return {"type": "instruction", "value": token}

        # Comment
        elif token == "%":
            self.advance()  # Ignore comments
            return {"type": "comment"}
        elif token == "#":
            self.advance()
            # '#' should be the last token, if not raise an error
            if self.current_token():
                self.errors.append({"error": "'#' must be the last instruction in the program.", "position": self.current_token_index})
            return {"type": "end_marker", "value": "#"}

        # Conditionals (si)
        elif token == "si":
            self.advance()
            condition = self.parse_condition()
            if condition not in ["0", "1"]:
                self.errors.append({"error": "Invalid condition in 'si' statement.", "position": self.current_token_index})
            inner_structure = []
            while self.current_token() != "}":
                if self.current_token() in ["boucle", "si"]:
                    inner_structure.append(self.parse_instruction())
                elif self.current_token() is None:
                    self.errors.append({"error": "Missing closing '}' for 'si' block", "position": self.current_token_index})
                    break
                else:
                    inner_structure.append(self.parse_instruction())
            if self.current_token() == "}":
                self.advance()
            else:
                self.errors.append({"error": "Expected '}' to close 'si' block", "position": self.current_token_index})
            return {"type": "si", "condition": condition, "content": inner_structure}

        # Loop (boucle)
        elif token == "boucle":
            self.boucle_stack.append("boucle")
            self.advance()
            inner_structure = []
            while self.current_token() != "}":
                if self.current_token() == "fin":
                    if not self.boucle_stack:
                        self.errors.append({"error": "'fin' outside of any 'boucle'", "position": self.current_token_index})
                    inner_structure.append({"type": "fin"})
                    self.advance()
                elif self.current_token() in ["boucle", "si"]:
                    inner_structure.append(self.parse_instruction())
                elif self.current_token() is None:
                    self.errors.append({"error": "Missing closing '}' for 'boucle' block", "position": self.current_token_index})
                    break
                else:
                    inner_structure.append(self.parse_instruction())
            if self.current_token() == "}":
                self.boucle_stack.pop()
                self.advance()
            else:
                self.errors.append({"error": "Expected '}' to close 'boucle' block", "position": self.current_token_index})
            return {"type": "boucle", "content": inner_structure}

        elif token == "fin":
            # 'fin' outside of any 'boucle' is an error
            if not self.boucle_stack:
                self.errors.append({"error": "'fin' outside of any 'boucle' block", "position": self.current_token_index})
            self.advance()
            return {"type": "fin"}

        else:
            # Unknown or invalid instruction
            self.errors.append({"error": f"Unknown instruction '{token}'", "position": self.current_token_index})
            self.advance()
            return {"type": "error", "value": f"unknown instruction '{token}'"}

    def parse_condition(self):
        """Parse condition in 'si' statement."""
        if self.current_token() == "(":
            self.advance()
            condition = self.current_token()
            self.advance()
            if self.current_token() == ")":
                self.advance()
                return condition
            else:
                self.errors.append({"error": "Expected ')' after condition in 'si'", "position": self.current_token_index})
        return None

In [146]:
# Charger et analyser le code depuis un fichier .ts
file_path = "err_programme_mtddv.ts" 

with open(file_path, "r", encoding="utf-8") as file:
    code = file.read()

parser = MTddVParser(code)
# Effectuer l'analyse
parsed_structure = parser.parse()

# Choisir le nom du fichier de sortie en fonction de la présence d'erreurs
if parser.errors:
    output_filename = "./results/parsed_structure_with_errors.json"
    # Afficher les erreurs à l'écran
    for error in parser.errors:
        print(f"Erreur détectée : {error['error']} à la position {error['position']}")
else:
    output_filename = "./results/parsed_structure_valid.json"

# Créer le contenu JSON
output = {
    "parsed_structure": parsed_structure,
    "errors": parser.errors
}

# Sauvegarder le contenu dans le fichier approprié
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(output, f, indent=4, ensure_ascii=False)

print(f"Parsed structure and detected errors have been saved to '{output_filename}'")

Erreur détectée : Invalid condition in 'si' statement. à la position 10
Erreur détectée : Missing closing '}' for 'si' block à la position 51
Erreur détectée : Expected '}' to close 'si' block à la position 51
Parsed structure and detected errors have been saved to './results/parsed_structure_with_errors.json'
